In [1]:
import tensorflow as tf
import os

gpus = tf.config.experimental.list_physical_devices('GPU')
os.environ["CUDA_VISIBLE_DEVICES"]="1" # specify which GPU to use
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

2021-09-23 16:12:31.900634: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-09-23 16:12:31.965859: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:5b:00.0 name: Quadro RTX 4000 computeCapability: 7.5
coreClock: 1.545GHz coreCount: 36 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 387.49GiB/s
2021-09-23 16:12:31.966322: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 1 with properties: 
pciBusID: 0000:9e:00.0 name: Quadro RTX 4000 computeCapability: 7.5
coreClock: 1.545GHz coreCount: 36 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 387.49GiB/s
2021-09-23 16:12:31.966549: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-09-23 16:12:31.968512: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-09

In [2]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.optim import optimize_acqf
from botorch.acquisition import UpperConfidenceBound

import time
import numpy as np
import matplotlib.pyplot as plt

device = "cpu"  # for now, set device to CPU as the number of samples are small.
dtype = torch.double
NOISE_SE = 0.0

In [3]:
# Function that loads the saved weights in CNN.

# input:
# path: hdf5 file that saves the weights of a VGG16 network.

# output:
# model: Keras model with VGG16 architecture and the loaded weights.
from keras import applications
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

def loadCNNmodel(path):
    model = applications.VGG16(weights=None, include_top=False, input_shape=(128, 128, 3))
    print('Model loaded')
    top_model = Sequential()
    top_model.add(Flatten(input_shape=model.output_shape[1:]))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dropout(0.0))
    top_model.add(Dense(1,activation=None))
    new_model = Sequential()

    for l in model.layers:
        new_model.add(l)

    new_model.add(top_model)
    new_model.load_weights(path)
    print('CNN model loaded with weights.')
    return new_model

Using TensorFlow backend.


In [4]:
model_path = '/home/chenyu/Desktop/CNNmeasurement/CoarseCNNtraining/VGG16/OptimalPar_newData/Test6_Normalize_HighCs_emitxdefocus/Attempt_03_emit+defocus_dropout_0.1/Final_model_2.h5'
model = loadCNNmodel(model_path)

2021-09-23 16:12:40.588960: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2021-09-23 16:12:40.625041: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2200000000 Hz
2021-09-23 16:12:40.629501: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f014c000b60 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-09-23 16:12:40.629539: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-09-23 16:12:40.883245: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55f9016c7f40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2021-09-23 16:12:40.883308: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Quadro RTX 4000, Compute Capability 7.5
2021-09-23 16:12:40.883321: I tensorflow

Model loaded
CNN model loaded with weights.


In [12]:
def scale_range(input, min, max):
    input += -(np.min(input))
    input /= np.max(input) / (max - min)
    input += min
    return input

In [95]:
# the function to get the output from GPT
# input:
# x: coordinate in the parameter space to be observed, 1-by-6 numpy array with each number between [0,1].
# model: Keras model that saves the CNN model to quantify the beam quality from GPT output.

# output:
# y: single number from CNN output, higher stands for better beam quality.
import sys
sys.path.insert(1, '/home/chenyu/Desktop/git/STEMalign/GPTrelated/')
from uscope_calc import sim

def get_GPT_output(x, model):
    x_list = []
    x_low = np.asarray([1000, -40, 387000, -685000, -3.7515e6, 119000, 640000])
    x_high = np.asarray([2800, 40, 393000, -622500, -3.7495e6, 120300, 651000])
    xlim, ylim, shadow = sim(
        alpha = 1.0e-4*5,
        S1 = 2.5e5,
        S2 = 2.5e5,
        H1 = x[0] * (x_high[0] - x_low[0]) + x_low[0],
        H2 = x[0] * (x_high[0] - x_low[0]) + x_low[0] + x[1] * (x_high[1] - x_low[1]) + x_low[1],
        S3 = x[4]* (x_high[5] - x_low[5]) + x_low[5],  # 119931.5,
        S4 = x[5]* (x_high[6] - x_low[6]) + x_low[6],  # 648691.415,
        S6 = x[2]* (x_high[2] - x_low[2]) + x_low[2],  # 390000,
        S7 = x[3]* (x_high[3] - x_low[3]) + x_low[3],  # -654100.0
        Obj = -3.7505e6,
    )
    frame = scale_range(shadow, 0, 1)
    new_channel = np.zeros(frame.shape)
    img_stack = np.dstack((frame, new_channel, new_channel))
    x_list.append(img_stack)
    x_list = np.concatenate([arr[np.newaxis] for arr in x_list])
    prediction = model.predict(x_list, batch_size = 1)
    # print(prediction)
    return shadow, 1 - prediction[0][0]

In [99]:
# Function that generate n random data point with noise.

# input:
# n: number of datapoints to gerate
def generate_initial_data(n):
    # generate training data
    train_x = torch.rand(n, 6, device=device, dtype=dtype)
    output_y = [get_GPT_output(np.array(train_x[i,:]), model)[1] for i in range(n)]
    train_y = torch.tensor(output_y).unsqueeze(-1)
    train_y = train_y + NOISE_SE * torch.randn_like(train_y) if NOISE_SE else train_y
    best_observed_value = train_x.max().item()
    return train_x, train_y

In [100]:
# initialize training data and the model
train_X, train_Y = generate_initial_data(n = 10)

In [57]:
gp = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_model(mll)
UCB = UpperConfidenceBound(gp, beta = 2)
bounds = torch.stack([torch.zeros(6, device = device), torch.ones(6, device = device)])
candidate, acq_value = optimize_acqf(
        UCB, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
    )

In [ ]:
best_seen_rep = []
time_list = []

for irep in range(1):
    
    train_X, train_Y = generate_initial_data(n = 10)

    gp = SingleTaskGP(train_X, train_Y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_model(mll)
    bounds = torch.stack([torch.zeros(6, device = device), torch.ones(6, device = device)])
    best_observed_value = []
    t0 = time.time()
    
    for iteration in range(10):

        fit_gpytorch_model(mll)

        UCB = UpperConfidenceBound(gp, beta = 2)    
        candidate, acq_value = optimize_acqf(
            UCB, bounds=bounds, q = 1, num_restarts=5, raw_samples=20,
        )
        new_x = candidate.detach()
        print(new_x)
        
        result = get_GPT_output(np.array(new_x[0]), model)
        new_y = torch.tensor(result[1]).unsqueeze(-1).unsqueeze(-1)
        new_y = new_y + NOISE_SE * torch.randn_like(new_y) if NOISE_SE else new_y
        train_X = torch.cat([train_X, new_x])
        train_Y = torch.cat([train_Y, new_y])
        best_value = np.array(new_y[0]) if not best_observed_value else max(np.array(new_y[0]), best_observed_value[-1])
        best_observed_value.append(best_value)

        # update GP model using dataset with new datapoint
        gp = SingleTaskGP(train_X, train_Y)
        mll = ExactMarginalLogLikelihood(gp.likelihood, gp)


        print(iteration, best_value)
        
    time_list.append(time.time() - t0)
    best_seen_rep.append(np.array(best_observed_value))

In [89]:
best_value = np.array(new_y[0]) if not best_observed_value else max(np.array(new_y[0]), best_observed_value[-1])
print(best_value)

[0.77919181]


In [79]:
torch.cat([train_Y, new_y.unsqueeze(-1)])

RuntimeError: torch.cat(): Tensors must have same number of dimensions: got 2 and 1

In [77]:
train_Y.shape
print(new_y.unsqueeze(-1).shape)

torch.Size([1])


In [ ]:
np.mean(time_list)